# Trial and Error

In [ ]:
# import subprocess

# # customMINIAOD generation
# CONFIGFILE = 'customMiniAODv2_cfg.py'
# NTHREADS   = 4
# INPUTFILE  = 'file:' + current_file_path
# OUTPUTFILE = INPUTFILE.replace('AODSIM', 'customMINIAODSIM')
# COMMAND = 'cmsRun -n ' + str(NTHREADS) + ' ' + CONFIGFILE + ' inputFiles=' + INPUTFILE + ' outputFile=' + OUTPUTFILE + ' maxEvents=-1'
# print(COMMAND)
# print('-'*80)


# logfile = OUTPUTFILE.replace('.root', '.log').replace('file:', '')
# f = open(logfile,'w')
# loglist.append(f)

# subprocess.Popen(COMMAND.split(' '), stderr=f)

In [ ]:
# EVCUTS   = {GenJet_pt[0]==200}
# SVCUTS   = 
# TRCUTS   = 
# SSAMPLES = 
# BSAMPLES = 
# PLOTNAME = 
# OUTLOG   = 
# NTHREADS = 4


In [ ]:
# operators = '><='
# cutdict_ev = {'RawMET' : 'RawMET_pt'}
# mystr = 'RawMET>=2.0'
# splitted = None
# for operator in operators:
#     if operator in mystr:
#         splitted = mystr.split(operator, 1)
#         splitted[0] = cutdict_ev[splitted[0]]
#         splitted[1] = operator + splitted[-1]
#         break
# print(''.join(splitted))

In [ ]:
import K_plotter as K
from types import SimpleNamespace
import ROOT

In [ ]:
from itertools import product

def grid_parameters(parameters):
    for params in product(*parameters.values()):
        yield dict(zip(parameters.keys(), params))

In [ ]:
# These will be the defaults abbreviations.

cutdict_ev = dict()
cutdict_sv = dict()
cutdict_tr = dict()


cutdict_ev['evPuppiMET'] = ['PuppiMET_pt',   '>', '200.']
cutdict_ev['evRawMET']   = ['RawMET_pt',     '>', '200.']
cutdict_ev['evGenMET']   = ['GenMET_pt',     '>', '200.']
cutdict_ev['evGenJet']   = ['GenJet_pt[0]',  '>', '100.']
cutdict_ev['evJet']      = ['Jet_pt[0]',     '>', '100.']


cutdict_sv["svNDOF"] =   ['SDVSecVtx_ndof',      '>', '1']
cutdict_sv['svRALPHA'] = ['SDVSecVtx_pAngle',    '>', '0.2']
cutdict_sv['svLXYSIG'] = ['SDVSecVtx_LxySig',    '>', '20']
cutdict_sv['svNGTR'] =   ['SDVSecVtx_newgoodtr', '>=', '2']
cutdict_sv['svDPHIRM'] = ['acos(cos(SDVSecVtx_L_phi-GenMET_phi))',     '<', '1.5']
cutdict_sv["svDPHIJ"] =  ['acos(cos(SDVSecVtx_L_phi-GenJet_phi[0]))',  '>', '1']


cutdict_tr['trDZ'] =     ['abs(SDVTrack_dz)',           '<', '4']
cutdict_tr['trCHI2'] =   ['SDVTrack_normalizedChi2',    '<', '5']
cutdict_tr['trNVH'] =    ['SDVTrack_numberOfValidHits', '>', '13']
cutdict_tr['trRSPT'] =   ['(SDVTrack_ptError/SDVTrack_pt)', '<', '0.015']
cutdict_tr['trDXYSIG'] = ['(abs(SDVTrack_dxy)/SDVTrack_dxyError)', '>', '4']
cutdict_tr["trDPHIJ"] =  ['acos(cos(SDVTrack_phi-GenJet_phi[0]))', '>', '1']
cutdict_tr["trDPHIM"] =  ['acos(cos(SDVTrack_phi-GenMET_phi))',    '<', '1.5']


# These will be the user's requests.

ev_cuts = 'evPuppiMET,evGenJet'.split(',')
sv_cuts = 'svNDOF,svRALPHA,svLXYSIG,svNGTR,svDPHIRM,svDPHIJ'.split(',')
tr_cuts = 'trDXYSIG,trCHI2,trDZ,trNVH,trRSPT,trDPHIJ,trDPHIM'.split(',')

ev_cuts = {key: value for key, value in cutdict_ev.items() if key in ev_cuts}
sv_cuts = {key: value for key, value in cutdict_sv.items() if key in sv_cuts}
tr_cuts = {key: value for key, value in cutdict_tr.items() if key in tr_cuts}

user_cuts_dict = {'ev': ev_cuts, 'sv': sv_cuts, 'tr': tr_cuts}
user_cuts = SimpleNamespace(**user_cuts_dict)


# These will be overriding the requests by the parameter grid to try different values.

grid_cuts = dict()
# grid_cuts['evPuppiMET'] = (100, 200, 400, 800)
grid_cuts['svLXYSIG'] = (5, 10, 20)
grid_cuts['svRALPHA'] = (0.1, 0.2, 0.3)
grid_cuts['trDZ'] = (2, 4, 8)


for grid in grid_parameters(grid_cuts):
    oride_lst = []
    for key, value in grid.items():
        
        if   key[:2] == 'ev':
            user_cuts.ev[key][-1] = str(value)
        elif key[:2] == 'sv':
            user_cuts.sv[key][-1] = str(value)
        elif key[:2] == 'tr':
            user_cuts.tr[key][-1] = str(value)
        else:
            raise ValueError('Follow the notation. (ev,sv,tr)')
    
        oride_lst.append(key)
        oride_lst.append(str(value))

    
    oride_str = '_'.join(oride_lst)
    
    cutstr_ev = '&&'.join([''.join(value) for key, value in user_cuts.ev.items()])
    cutstr_sv = '&&'.join([''.join(value) for key, value in user_cuts.sv.items()])
    cutstr_tr = '&&'.join([''.join(value) for key, value in user_cuts.tr.items()])

    cuts_dict = {'ev': cutstr_ev, 'sv': cutstr_sv, 'tr': cutstr_tr}
    cuts = SimpleNamespace(**cuts_dict)
    
    
    # Each time cuts will be different.
    # Now run the main code.

    var_and_bins = 'PuppiMET_pt,20,0,2000'
    
    print("Started main...")
    print('')
    print('var:   ', var_and_bins.split(',')[0])
    print('nbins: ', var_and_bins.split(',')[1])
    print('xlow:  ', var_and_bins.split(',')[2])
    print('xup:   ', var_and_bins.split(',')[3])
    print('')


    plot_args = K.DotDict()
    plot_args.title = 'Simulated Event Distributions After Cuts'
    plot_args.var =        var_and_bins.split(',')[0]
    plot_args.nb =     int(var_and_bins.split(',')[1])
    plot_args.xlow = float(var_and_bins.split(',')[2])
    plot_args.xup =  float(var_and_bins.split(',')[3])
    plot_args.savename = f'PLOTS/{plot_args.var}_{oride_str}.png'


    files = "/users/alikaan.gueven/samplesNewSeedNoDuplicates/customNANOAODSIM/STOP/600_588_200"
    sig_df, bkg_dfs = K.data_load_step(files)

    # Check if the var is entry level or subentry level (e.g. RawMET_pt or SDVSecVtx_Lxy)
    if sig_df.GetColumnType(plot_args.var)[:18] == 'ROOT::VecOps::RVec':
        df_ev, df_sv = K.masking_step(df, plot_args.var, cuts)
        hh_ev, hh_sv = K.histogram_step(df_ev, df_sv, plot_args, is_masked=True, debug=False)
        K.plotting_step(hh_ev, hh_sv, plot_args)

    else:
        sig_df_ev, sig_df_sv = K.filtering_step(sig_df, cuts)
        sig_hh_ev, sig_hh_sv = K.histogram_step(sig_df_ev, sig_df_sv, plot_args, debug=False)


        all_hh_sv = ROOT.TH1D('all_hh_sv', '', plot_args.nb, plot_args.xlow, plot_args.xup)
        
        for key, bkg_df in bkg_dfs.items():
            bkg_df_ev, bkg_df_sv = K.filtering_step(bkg_df, cuts)
            bkg_hh_ev, bkg_hh_sv = K.histogram_step(bkg_df_ev, bkg_df_sv, plot_args, debug=False)
            all_hh_sv.Add(all_hh_sv, bkg_hh_sv.GetPtr())
            
        all_hh_sv.Add(all_hh_sv, sig_hh_sv.GetPtr())

        K.tmp_plotting_step(all_hh_sv, sig_hh_sv, plot_args)

# Clean Code

In [1]:
def getCuts(ev, sv, tr):
    # These will be the defaults abbreviations.
    
    cutdict_ev = dict()
    cutdict_sv = dict()
    cutdict_tr = dict()
    
    
    cutdict_ev['evPuppiMET'] = ['PuppiMET_pt',   '>', '200.']
    cutdict_ev['evRawMET']   = ['RawMET_pt',     '>', '200.']
    cutdict_ev['evGenMET']   = ['GenMET_pt',     '>', '200.']
    cutdict_ev['evGenJet']   = ['GenJet_pt[0]',  '>', '100.']
    cutdict_ev['evJet']      = ['Jet_pt[0]',     '>', '100.']
    
    
    cutdict_sv["svNDOF"] =   ['SDVSecVtx_ndof',      '>', '1']
    cutdict_sv['svRALPHA'] = ['SDVSecVtx_pAngle',    '>', '0.2']
    cutdict_sv['svLXYSIG'] = ['SDVSecVtx_LxySig',    '>', '20']
    cutdict_sv['svNGTR'] =   ['SDVSecVtx_newgoodtr', '>=', '2']
    cutdict_sv['svDPHIRM'] = ['acos(cos(SDVSecVtx_L_phi-GenMET_phi))',     '<', '1.5']
    cutdict_sv["svDPHIJ"] =  ['acos(cos(SDVSecVtx_L_phi-GenJet_phi[0]))',  '>', '1']
    
    
    cutdict_tr['trDZ'] =     ['abs(SDVTrack_dz)',               '<', '4']
    cutdict_tr['trCHI2'] =   ['SDVTrack_normalizedChi2',        '<', '5']
    cutdict_tr['trNVH'] =    ['SDVTrack_numberOfValidHits',     '>', '13']
    cutdict_tr['trDXYDZ'] =  ['abs(SDVTrack_dxy/SDVTrack_dz)',  '>', '0']
    cutdict_tr['trRSPT'] =   ['(SDVTrack_ptError/SDVTrack_pt)', '<', '0.015']
    cutdict_tr['trDXYSIG'] = ['(abs(SDVTrack_dxy)/SDVTrack_dxyError)', '>', '4']
    cutdict_tr["trDPHIJ"] =  ['acos(cos(SDVTrack_phi-GenJet_phi[0]))', '>', '1']
    cutdict_tr["trDPHIM"] =  ['acos(cos(SDVTrack_phi-GenMET_phi))',    '<', '1.5']
    
    
    # These will be the user's requests.
    
    ev_cuts = ev.split(',')
    sv_cuts = sv.split(',')
    tr_cuts = tr.split(',')
    
    ev_cuts = {key: value for key, value in cutdict_ev.items() if key in ev_cuts}
    sv_cuts = {key: value for key, value in cutdict_sv.items() if key in sv_cuts}
    tr_cuts = {key: value for key, value in cutdict_tr.items() if key in tr_cuts}
    
    user_cuts_dict = {'ev': ev_cuts, 'sv': sv_cuts, 'tr': tr_cuts}
    user_cuts = SimpleNamespace(**user_cuts_dict)
    return user_cuts

In [2]:
def overrideCuts(user_cuts, override_dict):
    oride_lst = []
    print('Overrides:')
    for key, value in override_dict.items():
        if   key[:2] == 'ev':
            user_cuts.ev[key][-1] = str(value)
            print(''.join(user_cuts.ev[key]))
        elif key[:2] == 'sv':
            user_cuts.sv[key][-1] = str(value)
            print(''.join(user_cuts.sv[key]))
        elif key[:2] == 'tr':
            user_cuts.tr[key][-1] = str(value)
            print(''.join(user_cuts.tr[key]))
        else:
            raise ValueError('Follow the notation. (ev,sv,tr)')
    
        oride_lst.append(key)
        oride_lst.append(str(value))


    
    oride_str = '_'.join(oride_lst)
    
    cutstr_ev = '&&'.join([''.join(value) for key, value in user_cuts.ev.items()])
    cutstr_sv = '&&'.join([''.join(value) for key, value in user_cuts.sv.items()])
    cutstr_tr = '&&'.join([''.join(value) for key, value in user_cuts.tr.items()])

    cuts_dict = {'ev': cutstr_ev, 'sv': cutstr_sv, 'tr': cutstr_tr}
    cuts = SimpleNamespace(**cuts_dict)
    return cuts, oride_str

In [3]:
import K_plotter as K
from types import SimpleNamespace
import ROOT

Welcome to JupyROOT 6.28/00


In [4]:
import numpy as np

In [5]:
from itertools import product

def grid_parameters(parameters):
    for params in product(*parameters.values()):
        yield dict(zip(parameters.keys(), params))

In [ ]:
ev = 'evPuppiMET,evGenJet'
sv = 'svNDOF,svRALPHA,svLXYSIG,svNGTR,svDPHIRM,svDPHIJ'
# tr = 'trDXYSIG,trCHI2,trDZ,trNVH,trRSPT,trDPHIJ,trDPHIM'
tr = 'trCHI2,trNVH,trRSPT,trDPHIJ,trDPHIM,trDXYDZ'

user_cuts = getCuts(ev, sv, tr)


grid_cuts = dict()
# grid_cuts['evPuppiMET'] = (200, 400, 800)
grid_cuts['trDXYDZ'] = (0, 0.5, 1, 1.3, 1.6, 2, 2.5, 3, 3.5, 4)

for grid in grid_parameters(grid_cuts):
    print("ev_cuts: ", ev)
    print("sv_cuts: ", sv)
    print("tr_cuts: ", tr)
    print('')

    # print('Overrides:')
    # for key, value in grid.items():
    #     print('{:<12}'.format(key), value)
    # print('')

    cuts, oride_str = overrideCuts(user_cuts, grid)

    print('')
    print('cutstr_ev: ', cuts.ev)
    print('cutstr_sv: ', cuts.sv)
    print('cutstr_tr: ', cuts.tr)

    # Each time cuts will be different.
    # Now run the main code.

    var_and_bins = 'PuppiMET_pt,20,0,2000'
    
    print("Started main...")
    print('')
    print('var:   ', var_and_bins.split(',')[0])
    print('nbins: ', var_and_bins.split(',')[1])
    print('xlow:  ', var_and_bins.split(',')[2])
    print('xup:   ', var_and_bins.split(',')[3])
    print('')


    plot_args = K.DotDict()
    plot_args.title = 'Simulated Event Distributions After Cuts'
    plot_args.var =        var_and_bins.split(',')[0]
    plot_args.nb =     int(var_and_bins.split(',')[1])
    plot_args.xlow = float(var_and_bins.split(',')[2])
    plot_args.xup =  float(var_and_bins.split(',')[3])
    plot_args.savename = f'PLOTS/{plot_args.var}_{oride_str}.png'


    files = "/users/alikaan.gueven/samplesNewSeedNoDuplicates/customNANOAODSIM/STOP/600_588_200"
    sig_df, bkg_dfs = K.data_load_step(files)

    all_hh_sv = ROOT.TH1D('all_hh_sv', '', plot_args.nb, plot_args.xlow, plot_args.xup)

    # Check if the var is entry level or subentry level (e.g. RawMET_pt or SDVSecVtx_Lxy)
    if sig_df.GetColumnType(plot_args.var)[:18] == 'ROOT::VecOps::RVec':
        sig_df_ev, sig_df_sv = K.masking_step(sig_df, plot_args.var, cuts)
        sig_hh_ev, sig_hh_sv = K.histogram_step(sig_df_ev, sig_df_sv, plot_args, is_masked=True, debug=False)

        for key, bkg_df in bkg_dfs.items():
            bkg_df_ev, bkg_df_sv = K.masking_step(bkg_df, plot_args.var, cuts)
            bkg_hh_ev, bkg_hh_sv = K.histogram_step(bkg_df_ev, bkg_df_sv, plot_args, debug=False)
            all_hh_sv.Add(all_hh_sv, bkg_hh_sv.GetPtr())

    else:
        sig_df_ev, sig_df_sv = K.filtering_step(sig_df, cuts)
        sig_hh_ev, sig_hh_sv = K.histogram_step(sig_df_ev, sig_df_sv, plot_args, debug=False)

        for key, bkg_df in bkg_dfs.items():
            bkg_df_ev, bkg_df_sv = K.filtering_step(bkg_df, cuts)
            bkg_hh_ev, bkg_hh_sv = K.histogram_step(bkg_df_ev, bkg_df_sv, plot_args, debug=False)
            all_hh_sv.Add(all_hh_sv, bkg_hh_sv.GetPtr())

    
    all_hh_sv.Add(all_hh_sv, sig_hh_sv.GetPtr())
    K.tmp_plotting_step(all_hh_sv, sig_hh_sv, plot_args)

ev_cuts:  evPuppiMET,evGenJet
sv_cuts:  svNDOF,svRALPHA,svLXYSIG,svNGTR,svDPHIRM,svDPHIJ
tr_cuts:  trCHI2,trNVH,trRSPT,trDPHIJ,trDPHIM,trDXYDZ

Overrides:
abs(SDVTrack_dxy/SDVTrack_dz)>0

cutstr_ev:  PuppiMET_pt>200.&&GenJet_pt[0]>100.
cutstr_sv:  SDVSecVtx_ndof>1&&SDVSecVtx_pAngle>0.2&&SDVSecVtx_LxySig>20&&SDVSecVtx_newgoodtr>=2&&acos(cos(SDVSecVtx_L_phi-GenMET_phi))<1.5&&acos(cos(SDVSecVtx_L_phi-GenJet_phi[0]))>1
cutstr_tr:  SDVTrack_normalizedChi2<5&&SDVTrack_numberOfValidHits>13&&abs(SDVTrack_dxy/SDVTrack_dz)>0&&(SDVTrack_ptError/SDVTrack_pt)<0.015&&acos(cos(SDVTrack_phi-GenJet_phi[0]))>1&&acos(cos(SDVTrack_phi-GenMET_phi))<1.5
Started main...

var:    PuppiMET_pt
nbins:  20
xlow:   0
xup:    2000

ZJetsToNuNu_HT-100To200 84691
ZJetsToNuNu_HT-200To400 2307807
ZJetsToNuNu_HT-400To600 4285238
ZJetsToNuNu_HT-600To800 1350805
ZJetsToNuNu_HT-800To1200 632783
ZJetsToNuNu_HT-1200To2500 137901
ZJetsToNuNu_HT-2500ToInf 127421

nEvents:                  7533
nEvents passing ev cuts:  5593
nE